In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, os, shutil
import statistics
import json 
import requests
from difflib import SequenceMatcher
import sys
#sys.path.insert(1, '/home/marinedjaffardjy/Documents/wf_features/src/parsing')
#import parsing_snkmk as ps
#import get_characteristics as t
import json

In [2]:
def importing_json_files(file_wf):
    f_wf = open(file_wf) #informations for nf
    # returns JSON object as
    # a dictionary
    wf = json.load(f_wf)
    f_wf.close
    return wf

In [3]:
#importing the process info
proc_nf = importing_json_files('json/nf_all_proc.json')
proc_nf_tool = importing_json_files('json/nf_proc_tool.json')
proc_snk = importing_json_files('json/snk_rule_info.json')
proc_snk_tool = importing_json_files('json/snk_rule_info_tool.json')


In [4]:
#importing the wf and auth dict (github info)
dict_nf = importing_json_files('json/wf_new_crawl_nextflow.json')
auth_nf = importing_json_files('json/author_clem_nf.json')
dict_snk = importing_json_files('json/wf_crawl_snakemake.json')
auth_snk = importing_json_files('json/author_clem_snk.json')


### Most used tools

In [6]:
def find_unique_tools(proc_list):
    tools = []
    for proc in proc_list:
        for tool in proc["tools"]:
            tools.append(tool)
    return list(set(tools))

In [19]:
tools_nf = find_unique_tools(proc_nf_tool)
len(tools_nf)

641

In [20]:
tools_snk = find_unique_tools(proc_snk_tool)
len(tools_snk)

338

In [24]:
def infos_usage_tools(proc_list):
    tools = find_unique_tools(proc_list)
    df_tools = pd.DataFrame(columns=['toolname', 'nb_proc', 'list_proc', 'nb_wf', 'list_wf', 'nb_own', 'list_own'])
    for tool in tools:
        proc_t = []
        proc_nb = 0
        wf_t = []
        own_t = []
        for proc in proc_list:
            if(tool in proc["tools"]):
                proc_t.append(proc)
                proc_nb+=1
                wf_t.append(proc["wf_orig"])
                own_t.append(proc["owner"])
        df_tools = df_tools.append({'toolname': tool, 
                                    'nb_proc': proc_nb, 
                                    'list_proc': proc_t, 
                                    'nb_wf' :len(set(wf_t)), 
                                    'list_wf': set(wf_t), 
                                    'nb_own': len(set(own_t)), 
                                    'list_own': set(own_t)}, ignore_index = True)
    return df_tools.sort_values(by=['nb_wf'], ascending = False)

In [25]:
df_tools_nf = infos_usage_tools(proc_nf_tool)
df_tools_nf.head(20)

,toolname,nb_proc,list_proc,nb_wf,list_wf,nb_own,list_own
500,SAMtools,2128,"[{'owner': 'ABMicroBioinf', 'wf_orig': 'magph'...",562,"{isoseq, LG-PACT, covbench, rna_nf, benchmarki...",375,"{SJLeeY10, h3abionet, khigashi1987, elyadlezmi..."
48,FastQC,842,"[{'owner': 'ABMicroBioinf', 'wf_orig': 'magph'...",542,"{nf-core-radseq, 2018_coral_16S, new-rnaseq, n...",361,"{h3abionet, khigashi1987, sickle-in-africa, mo..."
384,MultiQC,779,"[{'owner': 'ABMicroBioinf', 'wf_orig': 'magph'...",517,"{nf-core-radseq, 2018_coral_16S, new-rnaseq, n...",307,"{h3abionet, khigashi1987, sickle-in-africa, mo..."
598,BWA,393,"[{'owner': 'ABMicroBioinf', 'wf_orig': 'nf-tb'...",206,"{mnaseseq, mgp1000, useful-nextflow-patterns, ...",157,"{SJLeeY10, h3abionet, sickle-in-africa, jianho..."
60,BCFtools,600,"[{'owner': 'ABMicroBioinf', 'wf_orig': 'nf-tb'...",169,"{nfvacal, strelka2-nf, mgp1000, table_annovar-...",116,"{h3abionet, sickle-in-africa, elyadlezmi, mahe..."
498,BEDTools,320,"[{'owner': 'ATpoint', 'wf_orig': 'atac_chip_pr...",156,"{csrna, mnaseseq, mgp1000, LG-PACT, nextflow-p...",129,"{salzmanlab, heuermh, lwratten, khigashi1987, ..."
473,Bowtie,291,"[{'owner': 'ABMicroBioinf', 'wf_orig': 'magph'...",127,"{csrna, pe_stacks_pipeline, as-mapping, rna_nf...",103,"{exsquire, erikrikarddaniel, khigashi1987, jia..."
531,GATK,993,"[{'owner': 'Akazhiel', 'wf_orig': 'NeoPred-NF'...",125,"{gatk, mgp1000, covbench, nf-core-gatkcohortca...",83,"{h3abionet, sickle-in-africa, elyadlezmi, DLBP..."
366,Picard,290,"[{'owner': 'ATpoint', 'wf_orig': 'atac_chip_pr...",121,"{csrna, mnaseseq, as-mapping, rna_nf, nf-dnase...",89,"{salzmanlab, csf-ngs, uct-cbio, mahesh-panchal..."
348,STAR,256,"[{'owner': 'Akazhiel', 'wf_orig': 'NeoPred-NF'...",120,"{as-mapping, icpipe-nextflow, rna_nf, clipseq....",99,"{guigolab, bhagesh-codebeast, elyadlezmi, mahe..."


In [26]:
df_tools_snk = infos_usage_tools(proc_snk_tool)
df_tools_snk.head(20)

,toolname,nb_proc,list_proc,nb_wf,list_wf,nb_own,list_own
272,SAMtools,1532,"[{'name': 'sort_alignments', 'code': '(name='s...",572,"{fcarelli/glre/47, filcfig/PCP/3, WatsonLab/Go...",290,"{Exenter, mdjaffardjy, philippmuench, ecalfee,..."
271,BEDTools,501,[{'name': 'intersect_crosslink_sites_with_peak...,194,"{bioxfu/iCLIP/1, alexzaccaron/2021_enec_mt/5, ...",108,"{vodkatad, mmontemurro, ecalfee, bioxfu, filcf..."
319,BWA,327,"[{'name': 'rRNAContamination', 'code': '(name=...",187,"{fcarelli/glre/5, alexzaccaron/2021_enec_mt/5,...",100,"{vodkatad, philippmuench, ecalfee, dfguan, rat..."
19,FastQC,236,"[{'name': 'fastqc', 'code': '(name='fastqc', l...",170,"{alneberg/2015-04-BARM-Concoct/1, bioxfu/iCLIP...",117,"{aya49, mdjaffardjy, vodkatad, ecalfee, UofABi..."
28,BCFtools,274,"[{'name': 'merge_vcf', 'code': '(name='merge_v...",124,"{leahkemp/testing_seattleflu_assembly/3, iqbal...",78,"{vodkatad, dfguan, Shmuppel, hevmarriott, AlBi..."
260,Bowtie,177,"[{'name': 'PhiXContamination', 'code': '(name=...",108,"{AlexanderMoerseburg/ATAC-Seq/1, leahkemp/test...",77,"{philippmuench, bioxfu, seb-mueller, iqbal-lab..."
215,MultiQC,137,"[{'name': 'multiqc', 'code': '(name='multiqc',...",100,"{pawel125/RNAseq_hpc_workflow/1, bioinformatic...",66,"{vodkatad, ecalfee, AAFC-BICoE, LUDC-bioinform..."
185,STAR,152,"[{'name': 'starAlignment', 'code': '(name='sta...",72,"{ahryho/snakemake-tutorial/8, mgildea87/RNAseq...",53,"{Exenter, mdjaffardjy, sztankatt, LUDC-bioinfo..."
91,Minimap2,118,"[{'name': 'create_bam', 'code': '(name='create...",71,"{bhattlab/bhattlab_workflows/8, jmeppley/np_re...",41,"{callumparr, jmeppley, AlBi-HHU, dfloresDIL, C..."
334,Cutadapt,81,"[{'name': 'Remove_sequencing_adapters', 'code'...",63,"{bioxfu/iCLIP/1, Hajibabaei-Lab/SCVUC_COI_meta...",44,"{jcmcnch, bioxfu, slebedeva, sterrettJD, dflor..."


### Role of the contributors

How many wf have been developped by the top 10/20 contributors ?
How many contributors have created only 1 wf ? How many wf have they created ?

In [34]:
proc_snk_tool[0]["wf_orig"]

'AAFC-BICoE/snakemake-barcoding-assembly-pipeline/1'

In [57]:
def get_list_wf(proc_list):
    list_wf = []
    for proc in proc_list:
        list_wf.append(proc["wf_orig"])
    return list(set(list_wf))

In [59]:
def get_list_wf_nf(proc_list):
    list_wf = []
    for proc in proc_list:
        list_wf.append(proc["owner"]+"/"+proc["wf_orig"])
    return list(set(list_wf))

In [51]:
def get_list_contrib_snk(wf_list, dict_wf):
    list_contrib = []
    for wf in wf_list:
        wf_name = wf.split("/")[0]+"/"+wf.split("/")[1]
        for el in dict_wf[wf_name]["contributors"]:
            list_contrib.append(el)
    return list(set(list_contrib))

In [70]:
def get_list_contrib_nf(wf_list, dict_wf):
    list_contrib = []
    for wf_name in wf_list:
        if(wf_name in dict_wf.keys()):
            for el in dict_wf[wf_name]["contributors"]:
                list_contrib.append(el)
    return list(set(list_contrib))

In [36]:
dict_nf['joshua-d-campbell/nf-GATK_Exome_Preprocess'].keys()

dict_keys(['files', 'name', 'id', 'git_url', 'update_date', 'creation_date', 'actual_date', 'last_push', 'owner', 'description', 'forks', 'stars', 'watchers', 'subscribers', 'languages', 'topics', 'archived', 'link', 'contributors', 'license'])

In [56]:
proc_nf_tool[0]["wf_orig"]

'magph'

In [84]:
def create_df_contrib_snk(proc_list, dict_wf):
    wf_list = get_list_wf(proc_list)
    contribs = get_list_contrib_snk(wf_list, dict_wf)
    df_contrib = pd.DataFrame(columns=['contributor', 'nb_wf', 'list_wf'])
    for contr in contribs:
        print(contr)
        list_wf = []
        for wf in wf_list:
            wf_name = wf.split("/")[0]+"/"+wf.split("/")[1]
            if(contr in dict_wf[wf_name]["contributors"]):
                list_wf.append(wf)
        df_contrib = df_contrib.append({'contributor':contr,
                                'nb_wf':len(set(list_wf)),
                                'list_wf':set(list_wf)}, ignore_index=True)
    return df_contrib.sort_values(by=['nb_wf'], ascending = False)

In [85]:
def create_df_contrib_nf(proc_list, dict_wf):
    wf_list = get_list_wf_nf(proc_list)
    contribs = get_list_contrib_nf(wf_list, dict_wf)
    df_contrib = pd.DataFrame(columns=['contributor', 'nb_wf', 'list_wf'])
    for contr in contribs:
        print(contr)
        list_wf = []
        for wf_name in wf_list:
            if(wf_name in dict_wf.keys()):
                if(contr in dict_wf[wf_name]["contributors"]):
                    list_wf.append(wf_name)
        print(list_wf)    
        df_contrib = df_contrib.append({'contributor':contr,
                                    'nb_wf':len(set(list_wf)),
                                    'list_wf':set(list_wf)}, ignore_index=True)
    return df_contrib.sort_values(by=['nb_wf'], ascending = False)

In [86]:
df_contrib_snk = create_df_contrib_snk(proc_snk_tool, dict_snk)

vodkatad
ecalfee
hevmarriott
filcfig
calchoo
BrandonJRomano
DCGenomics
rrodgers12
dcsoto
sndrtj
vsoch
JihedC
shigdon
HeyLifeHD
MuhammedHasan
aryarm
cjw85
Piezoid
martian-yan
JoeBlackSci
lig050129
yyolanda
elipapa
naveen584
MarineP-IODAA
MigleSur
c-mertes
susheelbhanu
danlooo
amberr098
liangyy
engreitz
swo
tesa1
NielInfante
CancerGenetics007
ksw9
daler
jiyideanjiao
laserson
glchn
EPLeyne
laseaman
dilworthlab
rmagoglia
willrosenow
elimoss
desireetillo
rarey
dfloresDIL
sorjuela
abwolf
waglecn
zhuchcn
hjzhou988
athalexiou
dmvandenberg
amholtz
Xavster838
JaneNjeri
MaleloAS
fionarhuang
fingerfen
Kanomble
mattdoug604
sbamin
GitStudentKanomble
jzehr
nataquinones
fburic
crazyhottommy
jimkwon
SchulzLab
wenweiliang
cuttlefishh
dvalenzu
maanmi
adadiehl
Tendo1434
akirosingh
sciccolella
Mass23
user611
trofimovamw
montpetitlab
smithowenk
silastittes
PaulineTurk
trvrb
skhilton
nuan604
dgmaghini
awilson0
d-henness
nicolerg
mnavascues
QiangTU
paulimer
mathbionerd
Tjitskedv
jeongdo801
jhavensHMC
natir
tw

In [87]:
df_contrib_snk.head(5)

,contributor,nb_wf,list_wf
46,elimoss,47,"{bhattlab/bhattlab_workflows/8, bhattlab/bhatt..."
283,bsiranosian,42,"{bhattlab/bhattlab_workflows/8, bhattlab/bhatt..."
309,jribado,40,"{bhattlab/bhattlab_workflows/8, bhattlab/bhatt..."
415,tamburinif,40,"{bhattlab/bhattlab_workflows/8, bhattlab/bhatt..."
374,navajain,40,"{bhattlab/bhattlab_workflows/8, bhattlab/bhatt..."


In [88]:
df_contrib_snk.to_csv("csv/df_contrib_snk.csv")

In [89]:
tot_wf_snk = len(get_list_wf(proc_snk_tool))

In [90]:
tot_20_snk = []
for wf_list in list(df_contrib_snk.head(20)["list_wf"]):
    for wf in wf_list:
        tot_20_snk.append(wf)
print(len(set(tot_20_snk)))
print(len(set(tot_20_snk))/tot_wf_snk)

223
0.17740652346857597


In [92]:
tot_10_snk = []
for wf_list in list(df_contrib_snk.head(10)["list_wf"]):
    for wf in wf_list:
        tot_10_snk.append(wf)
print(len(set(tot_10_snk)))
print(len(set(tot_10_snk))/tot_wf_snk)

133
0.10580747812251393


In [100]:
tot_1_snk = []
for wf_list in list(df_contrib_snk.loc[df_contrib_snk["nb_wf"]==1]["list_wf"]):
    for wf in wf_list:
        tot_1_snk.append(wf)
print(len(set(tot_1_snk)))
print(len(set(tot_1_snk))/tot_wf_snk)

299
0.23786793953858393


In [96]:
len(df_contrib_snk.loc[df_contrib_snk["nb_wf"]==1]["list_wf"])

395

In [97]:
len(df_contrib_snk)

714

In [98]:
len(df_contrib_snk.loc[df_contrib_snk["nb_wf"]==1]["list_wf"])/len(df_contrib_snk)

0.5532212885154062

In [101]:
df_contrib_nf = create_df_contrib_nf(proc_nf_tool, dict_nf)

bj8th
['sheynkman-lab/IsoSeq-Nextflow']
femiliani
['mckennalab/NextLineage']
KochTobi
['nf-core/hlatyping']
khigashi1987
['khigashi1987/CUTRUN_Nextflow']
DarianHole
['phac-nml/ncov-dehoster']
elyadlezmi
['elyadlezmi/RNA2CM']
winni2k
['markgene/cutnrun', 'jiangfuqing/Chip-seq', 'nf-core/chipseq']
lukfor
['genepi/nf-gwas', 'askimed/nf-test-examples']
monikaBrandt
['monikaBrandt/nf-core-mynewpipeline']
jbrestel
['VEuPathDB/proteinToGenomeAlignment']
rolivella
['elixir-cloud-proteomics-workflows/elixir_proteomics_QC', 'proteomicsunitcrg/qcloud2-pipeline']
DCGenomics
['NCBI-Hackathons/Nasty_Metagenomes']
idot
['csf-ngs/controldna']
vsoch
['marcodelapierre/demo-shpc-nf', 'GMS6804-master/rnatoy']
chrstphdm
['chrstphdm/nxf_course']
george-hall-ucl
['harleenduggal/nfcore-rnaseq', 'nf-core/rnaseq', 'harleenduggal/RNASEQ']
gsteinhardt
['GoalConsortium/goal_misc']
alvinwt
['icgc-argo-workflows/icgc-argo-sv-copy-number']
lukasjelonek
['lukasjelonek/sequencing-technology-usage-diagram']
angelovangel

jherrero
['UCL-BLIC/nf-ginkgo', 'UCL-BLIC/wgsalign']
bschiffthaler
['bschiffthaler/nf-radseq', 'bschiffthaler/nf-modules', 'bschiffthaler/nf-pseudopipe']
michaelbale
['michaelbale/JLab-IPSeqFlow', 'michaelbale/JLab-ATACFlow']
noirot
['nf-core/methylseq', 'FAANG/GSM-pipeline']
jimmyliu1326
['jimmyliu1326/SamnSero_Nextflow', 'jimmyliu1326/tCANS']
michieitel
['PalMuc/TransPi']
danrlu
['czbiohub/sc2-illumina-pipeline']
jermth
['jermth/hls-workshop']
jordeu
['nf-core/cutandrun']
jiangweiyao
['jiangweiyao/RefNAAP_nf', 'jiangweiyao/kraken2_biom_nf', 'jiangweiyao/quaisar_nf', 'jiangweiyao/paramsReadernf2', 'jiangweiyao/sra_mash_nf', 'jiangweiyao/metaphlan_nf', 'jiangweiyao/coregenome_align_nf']
ckandoth
['GoalConsortium/goal_misc']
gn5
['wtsi-hgi/nf_irods_to_lustre', 'wtsi-hgi/nf_scrna_deconvolution', 'wtsi-hgi/nextflow-pipelines', 'wtsi-hgi/nextflow_ci']
anwarMZ
['anwarMZ/nf-upcoast-v', 'cidgoh/cidgoh_qc']
FalkoHof
['Gregor-Mendel-Institute/isoseq3']
cagescudero
['PalMuc/TransPi']
bfssi-fores

fmaguire
['fmaguire/filter_flow', 'fmaguire/mag_binning_workflow', 'fmaguire/mag_sweep', 'fmaguire/diamond_workflow', 'bactopia/bactopia']
Thyra
['NAMlab/rnaseq-mapper']
mikek498
['mikek498/artic-gls', 'connor-lab/ncov2019-artic-nf']
mchimenti
['mchimenti/ATAC_seq_nextflow']
slimnsour
['jerdra/TIGR_PURR']
craigwindell
['eresearchqut/VirReport']
jguhlin
['jguhlin/useful-nextflow-patterns']
cirrina
['cirrina/ctg-rnaseq']
raygozag
['raygozag/rnaseq']
saramonzon
['nf-core/viralrecon', 'BU-ISCIII/SARS_Cov2_consensus-nf', 'rastiks/viralrecon', 'BU-ISCIII/panelLowFreq-nf']
leebrian
['CDCgov/mycosnp-nf']
martinghunt
['oxfordmmm/preprocessing']
kenyon-alexander
['ChadFibke/germ']
jhlegarreta
['scilus/tractoflow']
dvolk
['tiwarialok/new-rnaseq', 'davideyre/bug-flow']
joshua-d-campbell
['joshua-d-campbell/nf-GATK_Exome_Preprocess']
lars20070
['OpenMSConsulting/NextflowLFQ']
Niinleslie
['likelet/MesKit']
alakob
['alakob/ena-bracken-annotation']
marcodelapierre
['marcodelapierre/demo-shpc-nf', 'mar

Mxrcon
['Mxrcon/Savio_alignments_nf', 'nf-core/modules', 'Mxrcon/Gustavo_protein_anotation_nf', 'bactopia/bactopia', 'Mxrcon/Nextflow-study']
dianamarek
['biocorecrg/SIB_course_nextflow_Nov_2021']
LeahRoberts
['Vikash84/micropipe']
sminot
['FredHutch/ccs-nf', 'FredHutch/PGAP-nf', 'FredHutch/circulator-nf', 'FredHutch/index-genomes-nf', 'FredHutch/VMAA', 'FredHutch/prokka-nf', 'FredHutch/AMGMA', 'jgolob/maliampi', 'FredHutch/nf-sra', 'Golob-Minot/geneshot', 'FredHutch/nf-pacbio-bacterial', 'FredHutch/cellprofiler-batch-nf', 'FredHutch/flye-nf', 'FredHutch/diamond-build-tax-db', 'FredHutch/unicycler-nf', 'FredHutch/microbial-rnaseq']
esrice
['WarrenLab/longread-svs-nf', 'WarrenLab/shortread-polish-nf', 'esrice/workflows', 'WarrenLab/longread-polish-nf', 'WarrenLab/purge-haplotigs-nf']
vincenthhu
['vincenthhu/nf-core-westest']
imendes93
['lifebit-ai/pcgr-nf', 'lifebit-ai/bam2cram', 'imendes93/ReleaseTheKraken', 'lifebit-ai/ReleaseTheKraken', 'imendes93/sv-nf']
cdiener
['Gibbons-Lab/pipeli

k-florek
['wslh-bio/dryad', 'UPHL-BioNGS/Cecret', 'k-florek/mirkwater', 'wslh-bio/spriggan']
fethalen
['ThiloSchulze/eukaryotic-genome-assembly']
chenthorn
['zamanianlab/Core_RNAseq-nf']
priyanka-surana
['sanger-tol/readmapping']
malinlarsson
['sickle-in-africa/saw.sarek', 'nf-core/sarek', 'sripaladugu/germline_somatic', 'GMS6804-master/Sarek', 'Genomic-Medicine-Linkoping/nf-core-sarek', 'UMCUGenetics/sarek_ubec', 'melnel000/Sarek_CBIO']
turrisita
['AGROSAVIABIOUNIT/MT']
replikation
['RVanDamme/MUFFIN', 'nanozoo/wf_prepare_submission_dammit', 'replikation/blastn_so_hot', 'replikation/poreCov', 'replikation/docker_pipelines']
lmyiing
['xmuyulab/scRNAseq_pipelines']
helrick
['icgc-argo-workflows/icgc-argo-sv-copy-number']
finnor
['finnor/sample-renamer']
verku
['NBISweden/pipelines-nextflow']
fruce-ki
['ZuberLab/crispr-process-nf']
v-clarissa
['czbiohub/nf-tracer']
lekhakaranam
['czbiohub/nf-predictorthologs', 'czbiohub/nextflow-tutorial-2019']
agalitsyna
['agalitsyna/RedClib']
alemenze


alexandregilardet
['nf-core/eager', 'nf-core/bactmap', 'nf-core/modules']
lpantano
['egenesis/WF-NF-rnassembly', 'nf-core/smrnaseq', 'pilm-bioinformatics/pipelines-nf-genomes', 'pilm-bioinformatics/pipelines-nf-fastrnaseq', 'alam1988/Nextflow', 'junyu-boston/dicerna-rnaseq', 'harleenduggal/nfcore-rnaseq', 'pilm-bioinformatics/pipelines-nf-circtools', 'nf-core/rnaseq', 'harleenduggal/RNASEQ']
evanfloden
['lifebit-ai/kallisto-sra', 'nf-core/eager', 'tiwarialok/new-rnaseq', 'nextflow-io/nf-hack18', 'nextflow-io/rnaseq-nf', 'cbcrg/nf-phdcourse19', 'cbcrg/nf-phdcourse20', 'mageshwarm/rnaseq-nf-magesh', 'tobsecret/BADAS_Nextflow_Tutorial', 'seqeralabs/gatk4-germline-snps-indels', 'nextflow-io/nf-hack17-tutorial', 'cbcrg/nf-phdcourse21', 'evanfloden/nf-module-demo', 'lauramble/rnaseq-vizfada', 'pilm-bioinformatics/pipelines-nf-fastrnaseq', 'evanfloden/transXpress-nf', 'CRG-CNAG/CalliNGS-NF', 'Biocentric/rnaseq_debug', 'milescsmith/gcls-rnaseq-nf', 'nextflow-io/nfcamp-tutorial']
Lcornet
['Lcor

msmallegan
['msmallegan/rmghc-workshop-19']
lconde-ucl
['sickle-in-africa/saw.sarek', 'nf-core/sarek', 'sripaladugu/germline_somatic', 'Genomic-Medicine-Linkoping/nf-core-sarek', 'UMCUGenetics/sarek_ubec', 'UCL-BLIC/rnaseq_variant_calling', 'UCL-BLIC/wgsalign']
Lipinski-B
['IARCbioinfo/Imputation-nf', 'Monia234/IARC-imputation']
kaurravneet4123
['nibscbioinformatics/nf-core-conva', 'nibscbioinformatics/nf-core-viralevo', 'nf-core/modules']
thanhleviet
['food-safety-bioinformatics-hackathon/benchmarking-refseq', 'thanhleviet/nf-nanopore-assembly', 'nf-core/bactmap', 'nf-core/modules', 'connor-lab/ncov2019-artic-nf']
lassefolkersen
['nf-core/modules']
i-pletenev
['nf-core/modules']
dshafranskaya
['ablab/metaGT', 'ablab/graphamr']
veitveit
['veitveit/nf-core-wombatp']
Ichaffurin
['Ilchaffurin/Pipeline-RBMS']
mikecormier
['mikecormier/neoseq-seqcover-nf']
riederd
['icbi-lab/nextNEOpi', 'nf-core/modules']
peikovakate
['peikovakate/fastq']
kingzhuky
['ssun1116/meripseqpipe', 'kingzhuky/merip

Leon-Bichmann
['Leon-Bichmann/diaproteomics']
maxemil
['maxemil/PhyloMagnet']
alexandrebouchard
['UBC-Stat-ML/humi2']
garf0012
['garfinjm/legsta-nf']
apeltzer
['nf-core/bacass', 'nf-core/methylseq', 'cgpu/PGP-UK-sarek', 'sickle-in-africa/saw.sarek', 'nf-core/eager', 'markgene/cutnrun', 'jiangfuqing/Chip-seq', 'nf-core/sarek', 'nf-core/bactmap', 'nf-core/cookiecutter', 'FAANG/GSM-pipeline', 'nf-core/airrflow', 'tobsecret/covid19', 'nf-core/smrnaseq', 'sripaladugu/germline_somatic', 'maxibor/coproID', 'nf-core/vipr', 'Genomic-Medicine-Linkoping/nf-core-sarek', 'nf-core/mag', 'nf-core/cageseq', 'nf-core/chipseq', 'nf-core/exoseq', 'UMCUGenetics/sarek_ubec', 'lengfei5/atacseq_nf', 'nf-core/nascent', 'nf-core/scrnaseq', 'bhagesh-codebeast/nextflowdualrnaseq', 'nf-core/hlatyping', 'glormph/nf-core-dda-quant-proteomics', 'eQTL-Catalogue/rnaseq', 'nf-core/ampliseq', 'alam1988/Nextflow', 'nf-core/nanostring', 'junyu-boston/dicerna-rnaseq', 'nf-core/modules', 'harleenduggal/nfcore-rnaseq', 'jian

In [102]:
df_contrib_nf.to_csv("csv/df_contrib_nf.csv")

In [103]:
tot_wf_snk = len(get_list_wf(proc_snk_tool))
tot_20_snk = []
for wf_list in list(df_contrib_snk.head(20)["list_wf"]):
    for wf in wf_list:
        tot_20_snk.append(wf)
print(len(set(tot_20_snk)))
print(len(set(tot_20_snk))/tot_wf_snk)
tot_10_snk = []
for wf_list in list(df_contrib_snk.head(10)["list_wf"]):
    for wf in wf_list:
        tot_10_snk.append(wf)
print(len(set(tot_10_snk)))
print(len(set(tot_10_snk))/tot_wf_snk)
tot_1_snk = []
for wf_list in list(df_contrib_snk.loc[df_contrib_snk["nb_wf"]==1]["list_wf"]):
    for wf in wf_list:
        tot_1_snk.append(wf)
print(len(set(tot_1_snk)))
print(len(set(tot_1_snk))/tot_wf_snk)

223
0.17740652346857597
133
0.10580747812251393
299
0.23786793953858393


In [104]:
print(len(df_contrib_snk.loc[df_contrib_snk["nb_wf"]==1]["list_wf"]))
print(len(df_contrib_snk))
print(len(df_contrib_snk.loc[df_contrib_snk["nb_wf"]==1]["list_wf"])/len(df_contrib_snk))

395
714
0.5532212885154062


### Statistiques star, forks,...

In [109]:
list_wf_snk = [ el.split("/")[0]+"/"+el.split("/")[1] for el in get_list_wf(proc_snk_tool)]
list_wf_nf = get_list_wf_nf(proc_nf_tool)

In [107]:
dict_nf['joshua-d-campbell/nf-GATK_Exome_Preprocess'].keys()

dict_keys(['files', 'name', 'id', 'git_url', 'update_date', 'creation_date', 'actual_date', 'last_push', 'owner', 'description', 'forks', 'stars', 'watchers', 'subscribers', 'languages', 'topics', 'archived', 'link', 'contributors', 'license'])

In [112]:
def create_df_star_fork(list_wf, dict_wf):
    df_star = pd.DataFrame(columns=["name", "forks", "stars", "watchers", 'subscribers'])
    for wf in list_wf:
        if(wf in dict_wf.keys()):
            df_star = df_star.append({"name":wf, 
                                      "forks":dict_wf[wf]["forks"], 
                                      "stars":dict_wf[wf]["stars"], 
                                      "watchers":dict_wf[wf]["watchers"], 
                                      'subscribers':dict_wf[wf]["subscribers"]}, ignore_index=True)
    return df_star

In [113]:
df_star_snk = create_df_star_fork(list_wf_snk, dict_snk)

In [114]:
df_star_nf = create_df_star_fork(list_wf_nf, dict_nf)

In [117]:
def print_stats_forks(df_stars):
    print("Forks : moy "+str(df_stars["forks"].mean())+ " max "+str(df_stars["forks"].max()))
    print("Stars : moy "+str(df_stars["stars"].mean())+ " max "+str(df_stars["stars"].max()))
    print("Watchers : moy "+str(df_stars["watchers"].mean())+ " max "+str(df_stars["watchers"].max()))
    print("Subscribers : moy "+str(df_stars["subscribers"].mean())+ " max "+str(df_stars["subscribers"].max()))

In [118]:
print("Snk :")
print_stats_forks(df_star_snk)
print("Nf :")
print_stats_forks(df_star_nf)

Snk :
Forks : moy 1.4351630867143994 max 59
Stars : moy 3.443914081145585 max 208
Watchers : moy 3.443914081145585 max 208
Subscribers : moy 2.4184566428003182 max 32
Nf :
Forks : moy 2.4984177215189876 max 419
Stars : moy 3.1455696202531644 max 436
Watchers : moy 3.1455696202531644 max 436
Subscribers : moy 4.69620253164557 max 157


In [ ]:
df_star_snk.to_csv("csv/")